In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics.pairwise import cosine_similarity

# Load datasets
akg_bayi_df = pd.read_csv('https://raw.githubusercontent.com/Nourish-Path/MachineLearning/refs/heads/main/Dataset/AKG-bayi.csv')
food_check_df = pd.read_csv('https://raw.githubusercontent.com/Nourish-Path/MachineLearning/refs/heads/main/Dataset/food-check.csv')

# Inputs
try:
    age = int(input("Masukkan usia anak (dalam bulan): ").strip())
    num_foods = int(input("Berapa jumlah makanan yang ingin dimasukkan? ").strip())

    food_inputs = []
    for i in range(num_foods):
        print(f"Input makanan ke-{i + 1}")
        foods = input("Masukkan kategori makanan: ").strip()
        descriptions = input("Masukkan deskripsi makanan: ").strip()
        amount = float(input("Masukkan jumlah makanan (dalam gram atau ml): ").strip())
        food_inputs.append((foods, descriptions, amount))
except ValueError:
    raise ValueError("Input tidak valid, pastikan usia anak berupa angka dan jumlah makanan berupa angka desimal atau bulat.")

# Filter data based on input
akg_bayi_filtered = akg_bayi_df[akg_bayi_df['age'] == age]

if akg_bayi_filtered.empty:
    raise ValueError("Data tidak ditemukan untuk usia yang diberikan.")

selected_foods = []
for foods, descriptions, amount in food_inputs:
    filtered = food_check_df[
        (food_check_df['Category'] == foods) &
        (food_check_df['Description'] == descriptions)
    ]
    if filtered.empty:
        print(f"Data tidak ditemukan untuk kategori makanan '{foods}' dengan deskripsi '{descriptions}'.")
    else:
        # Adjust nutrients based on the amount
        adjusted_food = filtered.copy()
        adjusted_food.iloc[:, 2:] = (adjusted_food.iloc[:, 2:] * amount) / 100  # Nutrient columns start at 2
        selected_foods.append(adjusted_food)

if not selected_foods:
    raise ValueError("Tidak ada makanan yang sesuai dengan input.")

# Combine all selected foods into one DataFrame
food_check_filtered = pd.concat(selected_foods)

# Align column names for common nutrition metrics
common_columns = [col for col in akg_bayi_filtered.columns if col in food_check_filtered.columns and col != 'age']

# Ensure there's at least one common column
if not common_columns:
    raise ValueError("Tidak ada kolom nutrisi yang sesuai antara data AKG dan makanan.")

akg_bayi_filtered = akg_bayi_filtered[common_columns]
food_check_filtered = food_check_filtered[common_columns]

# Calculate nutrient differences
required_nutrients = akg_bayi_filtered.iloc[0].round(2)  # Nutrient required by the baby
consumed_nutrients = food_check_filtered.sum().round(2)  # Sum of nutrients from all foods
nutrient_differences = (required_nutrients - consumed_nutrients).round(2)

# Normalize nutrient data (for all foods)
scaler = MinMaxScaler()
normalized_food_data = pd.DataFrame(scaler.fit_transform(food_check_df[common_columns]),
                                    columns=common_columns)

# Normalize user's nutrient differences
normalized_nutrient_diff = scaler.transform(nutrient_differences.values.reshape(1, -1))

# Prepare the data for training
X = normalized_food_data.values
y = np.array([nutrient_differences.values] * len(X))

# Split data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the Sequential model
model = Sequential([
    Dense(128, input_dim=X.shape[1], activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(y.shape[1], activation='linear')  # Output layer: nutrisi yang direkomendasikan
])

# Compile the model
model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])

# Train the model
epochs = 100
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test))

# Get predictions for food recommendations
predictions = model.predict(X_test)

# Calculate cosine similarity between the user's nutrient differences and the food dataset
similarities = cosine_similarity(normalized_nutrient_diff, normalized_food_data)

# Add similarity scores to the food dataset
food_check_df['Relevance'] = similarities.flatten()

# Recommend top N foods
top_n = 5
recommended_foods = food_check_df.sort_values(by='Relevance', ascending=False).head(top_n)

# Display results
print("\nNutrisi yang harus dipenuhi:")
print(required_nutrients)

print("\nTotal nutrisi makanan yang telah dikonsumsi:")
print(consumed_nutrients)

print("\nSelisih nutrisi:")
print(nutrient_differences)

print("\nRekomendasi makanan berdasarkan Content-Based Filtering:")
print(recommended_foods[['Category', 'Description', 'Relevance']])

Masukkan usia anak (dalam bulan): 10
Berapa jumlah makanan yang ingin dimasukkan? 2
Input makanan ke-1
Masukkan kategori makanan: Milk
Masukkan deskripsi makanan: Milk, human
Masukkan jumlah makanan (dalam gram atau ml): 300
Input makanan ke-2
Masukkan kategori makanan: Banana pudding
Masukkan deskripsi makanan: Banana pudding
Masukkan jumlah makanan (dalam gram atau ml): 80
Epoch 1/100


<ipython-input-2-93119836ada8>:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    183.0
Name: Data.Vitamins.Vitamin A, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  adjusted_food.iloc[:, 2:] = (adjusted_food.iloc[:, 2:] * amount) / 100  # Nutrient columns start at 2
<ipython-input-2-93119836ada8>:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    96.0
Name: Data.Major Minerals.Calcium, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  adjusted_food.iloc[:, 2:] = (adjusted_food.iloc[:, 2:] * amount) / 100  # Nutrient columns start at 2
<ipython-input-2-93119836ada8>:47: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0    42.0
Name: Data.Major Minerals.Phos

178/178 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 24393.4219 - mae: 75.9591 - val_loss: 352.8989 - val_mae: 6.5447
Epoch 2/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 236.7733 - mae: 4.8488 - val_loss: 48.8203 - val_mae: 2.1182
Epoch 3/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 40.4085 - mae: 1.8369 - val_loss: 15.8640 - val_mae: 1.2739
Epoch 4/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 13.2382 - mae: 1.1762 - val_loss: 7.6875 - val_mae: 0.8963
Epoch 5/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 6.9392 - mae: 0.7950 - val_loss: 4.7053 - val_mae: 0.6622
Epoch 6/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.1843 - mae: 0.6392 - val_loss: 3.3765 - val_mae: 0.4698
Epoch 7/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5.2258 - mae: 0.5294 - val_loss: 2.6847 - val_mae: 0.4154
Epoch 8/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 3.8629 - mae: 0.4259 - val_loss: 2.3416 - val_mae: 0.3516
Epoch 9/100
178/178 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms